In [1]:
# Initial Import to get Databricks to start

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [2]:
import pandas as pd, numpy as np, re
from pathlib import Path
import datetime

from utilities import DeptRenamer, disp_all, fac_sk2id, get_active_store_ids

In [3]:
MIN_SQ_FT = 10000
LATEST_OPEN_DT = '2017-06-30'

active_store_list_fp = Path('../../../data/active_store_list.xlsx')
DF_DIR = Path('../../../data/temp_dfs/')

In [4]:
dept_renamer = DeptRenamer()

active_store_ids = get_active_store_ids(active_store_list_fp)

## Master Skel

In [5]:
m_df = pd.read_feather(DF_DIR/'skel_df.feather')

/anaconda3/envs/alb-shrink-databricks/lib/python3.5/site-packages/pandas/io/feather_format.py:112: FutureWarning: `nthreads` argument is deprecated, pass `use_threads` instead
  return feather.read_dataframe(path, nthreads=nthreads)


In [6]:
m_df.dropna(how='all', axis=1, inplace=True)

In [7]:
m_df['years_since_last_remodel'] = round((datetime.datetime.strptime('2018-12-31', '%Y-%m-%d') - m_df.last_remodel_dt).dt.days / 365, 2)

In [8]:
m_df['has_had_remodel'] = ~m_df['years_since_last_remodel'].isna()

In [9]:
m_df['years_since_last_remodel'].fillna(round((datetime.datetime.strptime('2018-12-31', '%Y-%m-%d') - m_df.opened_dt).dt.days / 365, 2), inplace=True)

In [10]:
m_df.head(10)

store_id                                  store_nm  region_id  \
0      1990  01990 FAC SANTA CLARA CA                          1   
1       978  00978 FAC FT BRAGG CA                             1   
2       970  00970 FAC SAN MATEO CA                            1   
3       928  00928 FAC PLEASANT HILL CA                        1   
4       983  00983 FAC LAKEPORT CA                             1   
5      1439  01439 FAC SUNNYVALE CA                            1   
6       774  00774 FAC BENICIA CA                              1   
7       917  00917 FAC WALNUT CREEK CA                         1   
8      3241  03241 FAC CAMPBELL CA                             1   
9      1794  01794 FAC FOLSOM CA                               1   

                                  region_nm  parent_op_area_id  \
0  WEST                                                     12   
1  WEST                                                     12   
2  WEST                                                     12   
3  WEST                                                     12   
4  WEST                                                     12   
5  WEST                                                     12   
6  WEST                                                     12   
7  WEST                                                     12   
8  WEST                                                     12   
9  WEST                                                     12   

                          parent_op_area_nm  op_area_finance_id  \
0  Northern California                                       45   
1  Northern California                                       45   
2  Northern California                                       45   
3  Northern California                                       45   
4  Northern California                                       45   
5  Northern California                                       45   
6  Northern California                                       45   
7  Northern California                                       45   
8  Northern California                                       45   
9  Northern California                                       45   

                         op_area_finance_nm  district_finance_id  \
0  Norcal SWY Retail Div #25                                  57   
1  Norcal SWY Retail Div #25                                  51   
2  Norcal SWY Retail Div #25                                  56   
3  Norcal SWY Retail Div #25                                  60   
4  Norcal SWY Retail Div #25                                  51   
5  Norcal SWY Retail Div #25                                  56   
6  Norcal SWY Retail Div #25                                  52   
7  Norcal SWY Retail Div #25                                  60   
8  Norcal SWY Retail Div #25                                  58   
9  Norcal SWY Retail Div #25                                  62   

   district_finance_cd       ...        never_dst_ind store_sub_type_cd  \
0                    7       ...                    N                 S   
1                    1       ...                    N                 S   
2                    6       ...                    N                 S   
3                   10       ...                    N                 S   
4                    1       ...                    N                 S   
5                    6       ...                    N                 S   
6                    2       ...                    N                 S   
7                   10       ...                    N                 S   
8                    8       ...                    N                 S   
9                   12       ...                    N                 S   

  non_lem_ind store_status_dt  store_status_id hours_from_host_tm  \
0           N      2017-10-31              2.0               -1.0   
1           N      2017-12-20              2.0               -1.0   
2      

## demos

In [11]:
!ls ../../../data/temp_dfs

fac_sale_com_txn_df.feather        store_demos.feather
shrink_customer_service_df.feather whse_shipment_item_df.feather
skel_df.feather


In [37]:
demos_df = pd.read_feather(DF_DIR/'store_demos.feather')

/anaconda3/envs/alb-shrink-databricks/lib/python3.5/site-packages/pandas/io/feather_format.py:112: FutureWarning: `nthreads` argument is deprecated, pass `use_threads` instead
  return feather.read_dataframe(path, nthreads=nthreads)


In [38]:
demos_df.rename(columns={'storenum' : 'store_id'}, inplace=True)

demos_df.drop(columns=['batch_id', 'radius', 'qtr', 'reid_', 'district'], inplace=True)

In [40]:
disp_all(demos_df)

division                    banner store_id  \
0          Acme                      Acme      114   
1          Acme                      Acme      147   
2          Acme                      Acme      192   
3          Acme                      Acme      291   
4          Acme                      Acme      292   
5          Acme                      Acme      293   
6          Acme                      Acme      294   
7          Acme                      Acme      295   
8          Acme                      Acme      299   
9          Acme                      Acme      613   
10         Acme                      Acme      664   
11         Acme                      Acme      703   
12         Acme                      Acme      721   
13         Acme                      Acme      726   
14         Acme                      Acme      730   
15         Acme                      Acme      731   
16         Acme                      Acme      769   
17         Acme                      Acme      773   
18         Acme                      Acme      776   
19         Acme                      Acme      778   
20         Acme                      Acme      779   
21         Acme                      Acme      780   
22         Acme                      Acme      787   
23         Acme                      Acme      806   
24         Acme                      Acme      808   
25         Acme                      Acme      809   
26         Acme                      Acme      813   
27         Acme                      Acme      817   
28         Acme                      Acme      820   
29         Acme                      Acme      821   
30         Acme                      Acme      826   
31         Acme                      Acme      828   
32         Acme                      Acme      831   
33         Acme                      Acme      833   
34         Acme                      Acme      845   
35         Acme                      Acme      849   
36         Acme                      Acme      852   
37         Acme                      Acme      859   
38         Acme                      Acme      871   
39         Acme                      Acme      872   
40         Acme                      Acme      874   
41         Acme                      Acme      882   
42         Acme                      Acme      890   
43         Acme                      Acme      893   
44         Acme                      Acme      896   
45         Acme                      Acme      943   
46         Acme                      Acme      949   
47         Acme                      Acme      992   
48         Acme                      Acme      996   
49         Acme                      Acme     1054   
50         Acme                      Acme     1058   
51         Acme                      Acme     1063   
52         Acme                      Acme     1065   
53         Acme                      Acme     1069   
54         Acme                      Acme     1077   
55         Acme                      Acme     1083   
56         Acme                      Acme     1092   
57         Acme                      Acme     1095   
58         Acme                      Acme     1096   
59         Acme                      Acme     1099   
60         Acme                      Acme     1582   
61         Acme                      Acme     1725   
62         Acme                      Acme     1744   
63         Acme                      Acme     1755   
64         Acme                      Acme     1764   
65         Acme                      Acme     1770   
66         Acme                      Acme     1777   
67         Acme                      Acme     1782   
68         Acme                      Acme     1787   
69         Acme                      Acme     1851   
70         Acme                      Acme     1856   
71         Acme                      Acme     1893   
72         Acme                      Acme     1908   
73         

In [41]:
demos_df.dtypes

division                object
banner                  object
store_id                object
address                 object
city                    object
st                      object
population               int64
growth_stability       float64
pcw                    float64
avail_groc               int64
seasonal_pop           float64
households               int64
avghh_size             float64
white_prcnt            float64
black_prcnt            float64
asian_prcnt            float64
hispanic_prcnt         float64
other_prcnt            float64
median_hh_income         int64
hhinc_lt_10k           float64
hhinc_10_15k           float64
hhinc_15_25k           float64
hhinc_25_34k           float64
hhinc_35_50k           float64
hhinc_50_75k           float64
hhinc_75_100k          float64
hhinc_100_125k         float64
hhinc_125_150k         float64
hhinc_150_175k         float64
hhinc_175_200k         float64
                        ...   
medianage              float64
age_lt_5

In [14]:
demos_df = demos_df.apply(lambda x: pd.to_numeric(x, errors='ignore'))

In [15]:
m_df = m_df.merge(demos_df, how='left', on='store_id')

store_id                                  store_nm  region_id  \
0         1990  01990 FAC SANTA CLARA CA                          1   
1          978  00978 FAC FT BRAGG CA                             1   
2          970  00970 FAC SAN MATEO CA                            1   
3          928  00928 FAC PLEASANT HILL CA                        1   
4          983  00983 FAC LAKEPORT CA                             1   
5         1439  01439 FAC SUNNYVALE CA                            1   
6          774  00774 FAC BENICIA CA                              1   
7          917  00917 FAC WALNUT CREEK CA                         1   
8         3241  03241 FAC CAMPBELL CA                             1   
9         1794  01794 FAC FOLSOM CA                               1   
10        2669  02669 FAC CARMEL CA                               1   
11        1895  01895 FAC FAIR OAKS CA                            1   
12        1866  01866 FAC ROCKLIN CA                              1   
13        2455  02455 FAC SAN ANSELMO CA                          1   
14         964  00964 FAC SAN FRANCISCO CA                        1   
15         936  00936 FAC WALNUT CREEK CA                         1   
16        3281  03281 FAC ALAMEDA CA                              1   
17        3111  03111 FAC SAN LEANDRO CA                          1   
18         305  00305 FAC REDWOOD CITY CA                         1   
19        1647  01647 FAC ELK GROVE CA                            1   
20         793  00793 FAC ARCATA CA                               1   
21        1824  01824 FAC SOUTH LAKE TAHOE CA                     1   
22        2621  02621 FAC BRENTWOOD CA                            1   
23        1109  01109 FAC VACAVILLE CA                            1   
24        2697  02697 FAC SACRAMENTO CA                           1   
25        1210  01210 FAC RENO NV                                 1   
26         995  00995 FAC SAN FRANCISCO CA                        1   
27        2255  02255 FAC HAWTHORNE NV                            1   
28        1954  01954 FAC REDDING CA                              1   
29        3124  03124 FAC MANTECA CA                              1   
...        ...                                       ...        ...   
2194      1268  01268 FAC SPOKANE WA                              1   
2195       462  00462 FAC YAKIMA WA                               1   
2196      3247  03247 FAC GRAND COULEE WA                         1   
2197        61  00061 FAC LAS VEGAS NV                            2   
2198       341  00341 FAC PAHRUMP NV                              2   
2199      4017  04017 FAC LAS VEGAS NV                            2   
2200       118  00118 FAC LAS VEGAS NV                            2   
2201      2613  02613 FAC LAS VEGAS NV                            2   
2202       276  00276 FAC LAS VEGAS NV                            2   
2203       129  00129 FAC LAS VEGAS NV                            2   
2204      2396  02396 FAC LAS VEGAS NV                            2   
2205      4005  04005 FAC NORTH LAS VEGAS NV                      2   
2206        89  00089 FAC LAS VEGAS NV                            2   
2207      1833  01833 FAC VALDEZ AK                               1   
2208       548  00548 FAC SOLDOTNA AK                             1   
2209      1802  01802 FAC ANCHORAGE AK                            1   
2210      1817  01817 FAC ANCHORAGE AK                            1   
2211      3252  03252 FAC MOSES LAKE WA                           1   
2212       265  00265 FAC SPOKANE WA                              1   
2213      1799  01799 FAC SPOKANE WA                              1   
2214       385  00385 FAC COLVILLE WA                             1   
2215       611  00611 FAC TEXARKANA AR                            2   
2216      1912  01912 FAC NEWPORT BEACH CA                        1   
2217      2803  02803 FAC SEAL BEACH CA                           1   
2218      2217  02217 FAC RANC